<a href="https://colab.research.google.com/github/Catherine-Nguyen88/project_chd/blob/main/DS3001_Project_2_Report_(Group_17).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**DS3001 Project 2 (Group 17)**

Isabella Dressel, Camila Gutierrez, Catherine Nguyen, Rhiannon Staley

**Summary**

Catherine

**Data**

The data used was from the Framingham Study which began in 1948 through the US Public Health Service in Massachusetts. 5,209 men and women were studied in this longitudinal investigation of what leads to cardiovascular disease (CVD). By further understanding the risk factors that lead to CVD, this research intends to highlight the incidence rates of disease. 32 clinical exams and an event follow-up collected data up until 2018. Our specific project focuses on building predictive algorithms that predict if an individual develops CHD. We used 8 dependent variables (sex, currentSmoker, cigsPerDay, diabetes, totChol, sysBP, diaBP, and BMO) and 1 independent variable (TenYearCHD) in our research. Sex is the is the recorded sex of the participant with 1 designating males, diabetes is whether the participant has dibetes as of the first exam, totChol is the total cholesterol (mg/dL), sysBP is systolic blood pressure (mmHg), and BMI is body mass index (weight (kg)/height (m)2. Finally, TenYearCHD was 1 when a patient developed coronary heart disease within 10 years of exam For all the variables, NAs were dropped. The timeline analyzed was 1948 to 2018 to contrast possible differences correlations in risk factors and developing CHD. The data were cleaned in Cleaning_Variables.ipynb and extracted in project1_extract_data.ipynb.

**Results**

Isabella and Rhiannon

**Conclusion**

Catherine

**Appendix**